In [4]:
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt

np.random.seed(0)

Dr = 300  # reservoir size
D = 3

mu = 1.2
lam = 0.000001

# Generate sparse reservoir matrix A1
A1 = sp.random(Dr, Dr, density=6/Dr, format='csr')
A1 = A1 - 0.5 * np.sign(A1.toarray())
A1 = A1 * mu / np.abs(sp.linalg.eigs(A1, k=1)[0])  # normalize reservoir matrix
A = sp.csr_matrix(A1)

Win = 0.01 * (2.0 * np.random.rand(Dr, D) - 1)

r = 0.5 - 1.0 * np.random.rand(Dr)
r0 = r.copy()

x, y, z = -2.9708, -5.1844, 12.1317

T = 100
dt = 0.02

ntraining = int(T / dt)

X = np.zeros((ntraining, D))
R = np.zeros((ntraining, Dr))

for t in range(ntraining):
    u = np.array([x, y, z])
    R[t, :] = r

    xp = 10 * (y - x)
    yp = x * (28 - z) - y
    zp = x * y - 2.66667 * z

    x += dt * xp
    y += dt * yp
    z += dt * zp

    r = np.tanh(A.dot(r) + Win.dot(u) + 1)
    X[t, :] = u

w = X.T.dot(R).dot(np.linalg.inv(R.T.dot(R) + lam * np.eye(Dr)))




In [14]:
X[4999]

array([-17.3207817 , -18.77841648,  38.77144311])

In [3]:
v = np.array([x, y, z])

nprediction = int(20 / dt)

xpre = np.zeros(nprediction)
ypre = np.zeros(nprediction)
zpre = np.zeros(nprediction)

xact = np.zeros(nprediction)
yact = np.zeros(nprediction)
zact = np.zeros(nprediction)

for t in range(nprediction):
    xp = 10 * (y - x)
    yp = x * (28 - z) - y
    zp = x * y - 2.66667 * z

    x += dt * xp
    y += dt * yp
    z += dt * zp

    r = np.tanh(A.dot(r) + Win.dot(v) + 1)
    v = w.dot(r)

    xpre[t] = v[0]
    ypre[t] = v[1]
    zpre[t] = v[2]

    xact[t] = x
    yact[t] = y
    zact[t] = z

times = dt * np.arange(1, nprediction + 1)

plt.figure()
plt.plot(times, xpre, 'o-', label='Predicted x')
plt.plot(times, xact, label='Actual x')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.show()

array([[ 0.13164493,  0.40771731,  0.28396287, ...,  0.45457626,
        -0.31913186, -0.32199318],
       [ 0.89500743,  0.86191401,  0.87146268, ...,  0.93259595,
         0.88023329,  0.8544009 ],
       [ 0.72306089,  0.9204488 ,  0.65705104, ...,  0.18697189,
         0.70786487,  0.98185675],
       ...,
       [ 0.79560092,  0.93115346, -0.26080908, ...,  0.36030115,
         0.55889305,  0.98140671],
       [ 0.79552993,  0.93352158, -0.24991549, ...,  0.37468892,
         0.54818179,  0.98256762],
       [ 0.79443307,  0.93465259, -0.23287497, ...,  0.38686367,
         0.53780074,  0.98398175]])